In [1]:
import time
cycle_time = time.time() #inizio a contare il tempo d' esecuzione
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from multiprocessing import Process


In [2]:
df = pd.read_csv('filled_age.csv') #apro il nuovo dataframe
df = df.drop(df.columns[0], axis=1) #rimuovo il vecchio indice

## Utilizzo una funzione per allenare in parallelo i classificatori

In [3]:
def runInParallel(*fns):
    proc = []
    for fn in fns:
        p = Process(target=fn)
        p.start()
        proc.append(p)
    for p in proc:
        p.join()

In [4]:
train, test = train_test_split(df, test_size=0.2)

In [5]:
label_train = train['Survived']
label_test = test['Survived']
train = train.drop('Survived', axis=1)
test = test.drop('Survived', axis=1)

In [6]:
def fitModelandScore(model, data, label, x, y):
    print(model.fit(data,label).score(x,y)*100)


In [7]:
for_cla = RandomForestClassifier(n_estimators=300, n_jobs=-1)
nn = MLPClassifier(hidden_layer_sizes=(20,20,20), alpha=0.001, max_iter=2500, activation='relu')
nn_bin = MLPClassifier(hidden_layer_sizes=(20,20,20), alpha=0.001, max_iter=2500, activation='logistic', solver='sgd', learning_rate_init=0.01,
                       )
sv = SVC(C=1, gamma=0.1, kernel='rbf')
gnb = GaussianNB()
ada = AdaBoostClassifier(n_estimators=300, learning_rate=0.01)

In [8]:
runInParallel(fitModelandScore(for_cla, train, label_train, test, label_test),
              fitModelandScore(nn, train, label_train, test, label_test),
              fitModelandScore(sv, train, label_train, test, label_test),
              fitModelandScore(gnb, train, label_train, test, label_test),
              fitModelandScore(ada, train, label_train, test, label_test),
              fitModelandScore(nn_bin, train, label_train, test, label_test),)

81.5642458101
79.3296089385
81.5642458101
78.7709497207
78.7709497207
62.5698324022
